# Taxis Data: Cleaning and Transforming

### 0. Premilinary Works
How big is the file, number of rows, etc.
### 1. Prepare a sample file

#### 0. Premilinary Works
How big is the file, number of rows, etc.

In [2]:
!ls -Rl
# 2017_Yellow_Taxi_Trip_Data.csv is 10532305815 bytes (9,80 GB)
# 2018_Yellow_Taxi_Trip_Data.csv is 10428263736 bytes (9,71 GB)
# 2019_Yellow_Taxi_Trip_Data.csv is 8197837930 bytes (7,63 GB)

.:
total 3660944
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rw-rw-rw- 1 angelrps angelrps   13400119 Jun 23 09:34 Data_Cleaned_To_Model.csv
-rw-rw-rw- 1 angelrps angelrps   24377425 Jun 20 06:26 Data_Taxis_2017_Cleaned.csv
-rw-rw-rw- 1 angelrps angelrps   26220204 Jun 20 04:20 Data_Taxis_2018_Cleaned.csv
-rw-rw-rw- 1 angelrps angelrps   26759445 Jun 20 00:08 Data_Taxis_2019_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps    3755058 Jun 22 10:01 NY_taxi_zones.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionar

In [4]:
# I am dealing with very heavy files so I will compress them (bz2) to work with them
# and remove the heavy .csv from my drive
#!bzip2 2017_Yellow_Taxi_Trip_Data.csv 2018_Yellow_Taxi_Trip_Data.csv 2019_Yellow_Taxi_Trip_Data.csv

In [5]:
!ls -Rl

.:
total 2897760
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionary_trip_records_yellow.pdf
-rwxrwxrwx 1 angelrps angelrps    1489310 Apr  7 19:34 taxi_zone_map_manhattan.jpg
-rwxrwxrwx 1 angelrps angelrps     202694 Apr  4 12:46 trip_record_user_guide.pdf


In [5]:
# Count Number of Lines: IT TAKES TIME, JUST BE AWARE BEFORE YOU UNCOMMENT
# !bzcat -c ./2019_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l
# !bzcat -c ./2018_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l
# !bzcat -c ./2017_Yellow_Taxi_Trip_Data.csv.bz2 | wc -l

# 2017_...113.496.875 lines
# 2018_...112.234.627 lines
# 2019_...84.399.020 lines
# The process took about 15 min to calculate the lines

84399020
112234627
113496875


In [11]:
!bzcat ./2019_Yellow_Taxi_Trip_Data.csv.bz2 | head -n 5

# Conclusions:
# 1. There is a header on the first line
# 2. The separator is ','

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,04/29/2019 11:31:03 AM,04/29/2019 11:31:03 AM,1,0,1,N,231,264,2,2.5,2.5,0.5,0,0,0.3,5.8,2.5
2,04/29/2019 10:52:08 AM,04/29/2019 11:06:18 AM,1,1.01,1,N,186,230,1,10,0,0.5,2.66,0,0.3,15.96,2.5
2,04/29/2019 11:28:40 AM,04/29/2019 11:33:01 AM,1,0.82,1,N,238,151,1,5,0,0.5,1.66,0,0.3,9.96,2.5
1,04/29/2019 11:28:06 AM,04/29/2019 12:07:32 PM,1,12.1,1,N,138,88,1,38.5,2.5,0.5,10.4,0,0.3,52.2,2.5

bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./2019_Yellow_Taxi_Trip_Data.csv.bz2, output file = (stdout)


### 1. Clean up dataset taking just a sample

In [1]:
# LOAD LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime as dt
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

The following cleaning actions are based on the dataset exploration made in the notebok 'Data_Analysis.ipynb'

1. Import part of the csv so the memory can handle it.
2. Let's remove trips under 0.06 (100m). I consider them either errors or not relevant
3. Remove negative 'fare_amount' or <1$. I consider them errors or not relevant.
4. Select the two columns of interest:
    - 'pickup_datetime'
    - 'PULocationID'
5. Drop unwanted years:<br>
Eventhough the dataset states the year, I have detected rows with wrong years so I´ll filter them out.
6. Groupby Hour and LocationID.
7. Break down datetime in other useful variables:
    - year
    - month
    - day
    - week
    - dayofweek
    - isweekend
    - isholiday

### Create Data Frame with Manhattan zones

In [2]:
# 1. Import Location and Borough columns form NY TAXI ZONES dataset
dfzones = pd.read_csv('../data/NY_taxi_zones.csv', sep=',',
                      usecols=['LocationID', 'borough'])

# 2. Filter Manhattan zones
    # Drop Borough column
    # Sort values
    # Drop duplicates
                #.drop(['borough'], axis=1)\
dfManZon = dfzones[dfzones['borough']=='Manhattan']\
            .sort_values(by='LocationID')\
            .drop_duplicates('LocationID').reset_index(drop=True)

dfManZon_array = dfManZon.iloc[:,0].values
print(dfManZon_array.size)
dfManZon_array

# There are 67 zones in Manhattan

67


array([  4,  12,  13,  24,  41,  42,  43,  45,  48,  50,  68,  74,  75,
        79,  87,  88,  90, 100, 103, 107, 113, 114, 116, 120, 125, 127,
       128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161,
       162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230,
       231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261,
       262, 263])

In [3]:
year = 2017
tu = pd.read_csv('../data/'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
tu = tu.rename(columns={'PULocationID':'LocationID'})
tu.head()

,pickup_datetime,trip_distance,LocationID,fare_amount
0,2017-06-14 17:04:21,0.56,237,4.5
1,2017-06-14 17:04:22,10.75,138,38.5
2,2017-06-14 17:04:22,1.83,75,10.0
3,2017-06-14 17:04:22,1.85,237,12.0
4,2017-06-14 17:04:22,2.25,239,18.5


In [25]:
# RECAP

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
year = 2017
dfsample = pd.read_csv('../data/'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

dfsample = dfsample.rename(columns={'PULocationID':'LocationID'})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)
# dfsample[dfsample['trip_distance']<0.06].index # this should be none

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)
#dfsample[dfsample['fare_amount']<1] # this should be none

# 4. Filter just Manhattan pickups
dfsample = dfsample.loc[dfsample['LocationID'].isin(dfManZon_array)]
#dfsample['PULocationID'].unique().shape

# 5. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','LocationID']].copy()
#dfsample.head()

# 6. Drop unwanted years
dfsample.drop(dfsample[dfsample['pickup_datetime'] < pd.Timestamp(dt(year,1,1))].index, inplace=True)
dfsample.drop(dfsample[dfsample['pickup_datetime'] >= pd.Timestamp(dt(year+1,1,1))].index, inplace=True)

# 7. Groupby HOUR and LocationID
    # set time period index
dfsample.set_index('pickup_datetime', inplace=True) # Set timestamp as index
dfsample = dfsample.to_period("H") # convert DATE to 1 hour PERIODS
    # Sort by datetime and by Location
dfsample.sort_values(['pickup_datetime', 'LocationID'], inplace=True)
    # Create new column to count Number of Pickups per LocationID
dfsample['NoOfPickups'] =1
    # Groupby Datetime and Location to get Pickups sum()
dfsample=dfsample.groupby(['pickup_datetime', 'LocationID'])['NoOfPickups'].sum().reset_index()
#dfsample.head(50)

# 8. Create new variables from datetime
dfsample['year'] = dfsample['pickup_datetime'].dt.year
dfsample['month'] = dfsample['pickup_datetime'].dt.month
dfsample['day'] = dfsample['pickup_datetime'].dt.day
dfsample['hour'] = dfsample['pickup_datetime'].dt.hour
dfsample['week'] = dfsample['pickup_datetime'].dt.week
dfsample['dayofweek'] = dfsample['pickup_datetime'].dt.dayofweek
    # 8.1 create variable 'isweekend'
mask = (dfsample['dayofweek'] == 5) | (dfsample['dayofweek'] == 6)
dfsample['isweekend'] = np.where(mask, 1, 0)
#dfsample.sample(50)

    # 8.2 Create date time index calendar
drange = pd.date_range(start=str(year)+'-01-01', end=str(year)+'-12-31')
cal = calendar()
holidays = cal.holidays(start=drange.min(), end=drange.max())
    # 8.3 create new column 'isholiday'
dfsample['isholiday'] = dfsample['pickup_datetime'].isin(holidays).astype(int)

print (dfsample.tail())
print(dfsample.info())
print(holidays)

     pickup_datetime  LocationID  NoOfPickups  year  month  day  hour  week  \
56  2017-06-14 17:00         246          145  2017      6   14    17    24   
57  2017-06-14 17:00         249          117  2017      6   14    17    24   
58  2017-06-14 17:00         261           53  2017      6   14    17    24   
59  2017-06-14 17:00         262          126  2017      6   14    17    24   
60  2017-06-14 17:00         263          160  2017      6   14    17    24   

    dayofweek  isweekend  isholiday  
56          2          0          0  
57          2          0          0  
58          2          0          0  
59          2          0          0  
60          2          0          0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 11 columns):
pickup_datetime    61 non-null period[H]
LocationID         61 non-null int64
NoOfPickups        61 non-null int64
year               61 non-null int64
month              61 non-null int64
day  

In [5]:
dfsample.shape

(61, 11)

## 2. Test Big Data approach on a sample

### DEFINE FUNCTION TO CLEAN DATASET
### DEFINE FUNCTION TO ADD DATE FEATURES TO DATASET

In [26]:
# DEFINE FUNCTION TO CLEAN DATASET
def clean_data(dfinput, yearinput, manhattan_zones_array):
    
    dfinput = dfinput.rename(columns={'PULocationID':'LocationID'})
    DistIndex_to_drop = dfinput[dfinput['trip_distance'] < 0.06].index
    dfinput.drop(DistIndex_to_drop, inplace=True)

    FareIndex_to_drop = dfinput[dfinput['fare_amount']<1].index
    dfinput.drop(FareIndex_to_drop, inplace=True)
    
    dfinput = dfinput.loc[dfinput['LocationID'].isin(manhattan_zones_array)]

    dfinput = dfinput[['pickup_datetime','LocationID']].copy()

    dfinput.drop(dfinput[dfinput['pickup_datetime'] < pd.Timestamp(dt(yearinput,1,1))].index, inplace=True)
    dfinput.drop(dfinput[dfinput['pickup_datetime'] >= pd.Timestamp(dt(yearinput+1,1,1))].index, inplace=True)

    dfinput.set_index('pickup_datetime', inplace=True) # Set timestamp as index
    dfinput = dfinput.to_period("H") # convert DATE to 1 hour PERIODS
    dfinput.sort_values(['pickup_datetime', 'LocationID'], inplace=True)
    dfinput['NoOfPickups'] =1
    
    dfinput=dfinput.groupby(['pickup_datetime', 'LocationID'])['NoOfPickups'].sum().reset_index()
    
    return dfinput

# DEFINE FUNCTION TO ADD DATE FEATURES TO DATASET
def add_date_features(dfinput, yearinput):
    
    dfinput['year'] = dfinput['pickup_datetime'].dt.year
    dfinput['month'] = dfinput['pickup_datetime'].dt.month
    dfinput['day'] = dfinput['pickup_datetime'].dt.day
    dfinput['hour'] = dfinput['pickup_datetime'].dt.hour
    dfinput['week'] = dfinput['pickup_datetime'].dt.week
    dfinput['dayofweek'] = dfinput['pickup_datetime'].dt.dayofweek

    mask = (dfinput['dayofweek'] == 5) | (dfinput['dayofweek'] == 6)
    dfinput['isweekend'] = np.where(mask, 1, 0)

    drange = pd.date_range(start=str(yearinput)+'-01-01', end=str(yearinput)+'-12-31')
    cal = calendar()
    holidays = cal.holidays(start=drange.min(), end=drange.max())
        # 6.3 create new column 'isholiday'
    dfinput['isholiday'] = dfinput['pickup_datetime'].isin(holidays).astype(int)
    return dfinput

In [7]:
# TEST FUNCTIONS
# import file
year = 2017
dfsample_test = pd.read_csv('../data/'+str(year)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

dfsample_test = clean_data(dfsample_test, year, dfManZon_array)
dfsample_test = add_date_features(dfsample_test, year)
dfsample_test.head()

,pickup_datetime,LocationID,NoOfPickups,year,month,day,hour,week,dayofweek,isweekend,IsHoliday
0,2017-06-14 17:00,4,12,2017,6,14,17,24,2,0,0
1,2017-06-14 17:00,12,2,2017,6,14,17,24,2,0,0
2,2017-06-14 17:00,13,71,2017,6,14,17,24,2,0,0
3,2017-06-14 17:00,24,30,2017,6,14,17,24,2,0,0
4,2017-06-14 17:00,41,35,2017,6,14,17,24,2,0,0


### FUNCTION TO RUN FOR EACH YEAR DATASET

In [8]:
# FUNCTION TO RUN FOR EACH YEAR DATASET

def process_data_sample(yearIN, manhattan_zones_array):
    
    # 1. Get iterator with 3000 chunksize
    dfsi = pd.read_csv('../data/'+str(yearIN)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       chunksize=3000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
    
    # 2. Declare empty DataFrame to store results of each chunk
    all_chunks=pd.DataFrame()

    # 3. Iterate through chunks and append results
    for i, dfs in enumerate(dfsi):
        print(i, len(dfs))

        # clean dataset
        result = clean_data(dfs, yearIN, manhattan_zones_array)

        all_chunks=all_chunks.append(result)

    # The chunks may be just part of a single day so I need to sort and groupby again
    result_all = all_chunks.sort_values(['pickup_datetime', 'LocationID']). \
                                        groupby(['pickup_datetime', 'LocationID'])['NoOfPickups'].sum().reset_index()
    # add date features to dataset
    add_date_features(result_all, yearIN)
    
    # save csv
    result_all.to_csv('../data/Data_Taxis_'+str(yearIN)+'_Cleaned_TEST.csv', index = False, header=True)
    return result_all

In [9]:
dfsample_test = process_data_sample(2017,dfManZon_array)
dfsample_test.head(10)

0 3000
1 3000
2 3000
3 999


,pickup_datetime,LocationID,NoOfPickups,year,month,day,hour,week,dayofweek,isweekend,IsHoliday
0,2017-06-14 17:00,4,12,2017,6,14,17,24,2,0,0
1,2017-06-14 17:00,12,2,2017,6,14,17,24,2,0,0
2,2017-06-14 17:00,13,71,2017,6,14,17,24,2,0,0
3,2017-06-14 17:00,24,30,2017,6,14,17,24,2,0,0
4,2017-06-14 17:00,41,35,2017,6,14,17,24,2,0,0
5,2017-06-14 17:00,42,21,2017,6,14,17,24,2,0,0
6,2017-06-14 17:00,43,301,2017,6,14,17,24,2,0,0
7,2017-06-14 17:00,45,40,2017,6,14,17,24,2,0,0
8,2017-06-14 17:00,48,238,2017,6,14,17,24,2,0,0
9,2017-06-14 17:00,50,80,2017,6,14,17,24,2,0,0


I check that the 'dfsample_test' dataframe and 'dfsample' are the same:
- Shape.
- Head and Tail.
- Taking a random row, they should be the same.

In [10]:
print(dfsample.shape)
print(dfsample_test.shape)

(61, 11)
(61, 11)


In [11]:
print(dfsample.tail())
print(dfsample_test.tail())

     pickup_datetime  LocationID  NoOfPickups  year  month  day  hour  week  \
56  2017-06-14 17:00         246          145  2017      6   14    17    24   
57  2017-06-14 17:00         249          117  2017      6   14    17    24   
58  2017-06-14 17:00         261           53  2017      6   14    17    24   
59  2017-06-14 17:00         262          126  2017      6   14    17    24   
60  2017-06-14 17:00         263          160  2017      6   14    17    24   

    dayofweek  isweekend  IsHoliday  
56          2          0          0  
57          2          0          0  
58          2          0          0  
59          2          0          0  
60          2          0          0  
     pickup_datetime  LocationID  NoOfPickups  year  month  day  hour  week  \
56  2017-06-14 17:00         246          145  2017      6   14    17    24   
57  2017-06-14 17:00         249          117  2017      6   14    17    24   
58  2017-06-14 17:00         261           53  2017      6  

In [13]:
print(dfsample.head())
print(dfsample_test.head())

    pickup_datetime  LocationID  NoOfPickups  year  month  day  hour  week  \
0  2017-06-14 17:00           4           12  2017      6   14    17    24   
1  2017-06-14 17:00          12            2  2017      6   14    17    24   
2  2017-06-14 17:00          13           71  2017      6   14    17    24   
3  2017-06-14 17:00          24           30  2017      6   14    17    24   
4  2017-06-14 17:00          41           35  2017      6   14    17    24   

   dayofweek  isweekend  IsHoliday  
0          2          0          0  
1          2          0          0  
2          2          0          0  
3          2          0          0  
4          2          0          0  
    pickup_datetime  LocationID  NoOfPickups  year  month  day  hour  week  \
0  2017-06-14 17:00           4           12  2017      6   14    17    24   
1  2017-06-14 17:00          12            2  2017      6   14    17    24   
2  2017-06-14 17:00          13           71  2017      6   14    17    24 

In [14]:
# CHECK A RANDOM ROW
random_row=random.randint(1,len(dfsample.index))
print(dfsample.loc[random_row])
print(dfsample_test.loc[random_row])

pickup_datetime    2017-06-14 17:00
LocationID                       88
NoOfPickups                      47
year                           2017
month                             6
day                              14
hour                             17
week                             24
dayofweek                         2
isweekend                         0
IsHoliday                         0
Name: 15, dtype: object
pickup_datetime    2017-06-14 17:00
LocationID                       88
NoOfPickups                      47
year                           2017
month                             6
day                              14
hour                             17
week                             24
dayofweek                         2
isweekend                         0
IsHoliday                         0
Name: 15, dtype: object


### 3. Run program with Big Data

In [18]:
# FUNCTION TO RUN FOR EACH YEAR DATASET

def process_data(yearIN, manhattan_zones_array):
    
    # 1. Get iterator with 3000 chunksize
    dfsi = pd.read_csv('../data/'+str(yearIN)+'_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       chunksize=3000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
    
    # 2. Declare empty DataFrame to store results of each chunk
    all_chunks=pd.DataFrame()

    # 3. Iterate through chunks and append results
    for i, dfs in enumerate(dfsi):
        print(i, len(dfs))

        # clean dataset
        result = clean_data(dfs, yearIN, manhattan_zones_array)

        all_chunks=all_chunks.append(result)

    # The chunks may be just part of a single day so I need to sort and groupby again
    result_all = all_chunks.sort_values(['pickup_datetime', 'LocationID']). \
                                        groupby(['pickup_datetime', 'LocationID'])['NoOfPickups'].sum().reset_index()
    # add date features to dataset
    add_date_features(result_all, yearIN)
    
    # save csv
    result_all.to_csv('../data/Data_Taxis_'+str(yearIN)+'_Cleaned.csv', index = False, header=True)

In [19]:
process_data(2017,dfManZon_array)

0 3000
1 3000
2 3000
3 3000
4 3000
5 3000
6 3000
7 3000
8 3000
9 3000
10 3000
11 3000
12 3000
13 3000
14 3000
15 3000
16 3000
17 3000
18 3000
19 3000
20 3000
21 3000
22 3000
23 3000
24 3000
25 3000
26 3000
27 3000
28 3000
29 3000
30 3000
31 3000
32 3000
33 3000
34 3000
35 3000
36 3000
37 3000
38 3000
39 3000
40 3000
41 3000
42 3000
43 3000
44 3000
45 3000
46 3000
47 3000
48 3000
49 3000
50 3000
51 3000
52 3000
53 3000
54 3000
55 3000
56 3000
57 3000
58 3000
59 3000
60 3000
61 3000
62 3000
63 3000
64 3000
65 3000
66 3000
67 3000
68 3000
69 3000
70 3000
71 3000
72 3000
73 3000
74 3000
75 3000
76 3000
77 3000
78 3000
79 3000
80 3000
81 3000
82 3000
83 3000
84 3000
85 3000
86 3000
87 3000
88 3000
89 3000
90 3000
91 3000
92 3000
93 3000
94 3000
95 3000
96 3000
97 3000
98 3000
99 3000
100 3000
101 3000
102 3000
103 3000
104 3000
105 3000
106 3000
107 3000
108 3000
109 3000
110 3000
111 3000
112 3000
113 3000
114 3000
115 3000
116 3000
117 3000
118 3000
119 3000
120 3000
121 3000
122 3000
123

924 3000
925 3000
926 3000
927 3000
928 3000
929 3000
930 3000
931 3000
932 3000
933 3000
934 3000
935 3000
936 3000
937 3000
938 3000
939 3000
940 3000
941 3000
942 3000
943 3000
944 3000
945 3000
946 3000
947 3000
948 3000
949 3000
950 3000
951 3000
952 3000
953 3000
954 3000
955 3000
956 3000
957 3000
958 3000
959 3000
960 3000
961 3000
962 3000
963 3000
964 3000
965 3000
966 3000
967 3000
968 3000
969 3000
970 3000
971 3000
972 3000
973 3000
974 3000
975 3000
976 3000
977 3000
978 3000
979 3000
980 3000
981 3000
982 3000
983 3000
984 3000
985 3000
986 3000
987 3000
988 3000
989 3000
990 3000
991 3000
992 3000
993 3000
994 3000
995 3000
996 3000
997 3000
998 3000
999 3000
1000 3000
1001 3000
1002 3000
1003 3000
1004 3000
1005 3000
1006 3000
1007 3000
1008 3000
1009 3000
1010 3000
1011 3000
1012 3000
1013 3000
1014 3000
1015 3000
1016 3000
1017 3000
1018 3000
1019 3000
1020 3000
1021 3000
1022 3000
1023 3000
1024 3000
1025 3000
1026 3000
1027 3000
1028 3000
1029 3000
1030 3000
1031 3

1751 3000
1752 3000
1753 3000
1754 3000
1755 3000
1756 3000
1757 3000
1758 3000
1759 3000
1760 3000
1761 3000
1762 3000
1763 3000
1764 3000
1765 3000
1766 3000
1767 3000
1768 3000
1769 3000
1770 3000
1771 3000
1772 3000
1773 3000
1774 3000
1775 3000
1776 3000
1777 3000
1778 3000
1779 3000
1780 3000
1781 3000
1782 3000
1783 3000
1784 3000
1785 3000
1786 3000
1787 3000
1788 3000
1789 3000
1790 3000
1791 3000
1792 3000
1793 3000
1794 3000
1795 3000
1796 3000
1797 3000
1798 3000
1799 3000
1800 3000
1801 3000
1802 3000
1803 3000
1804 3000
1805 3000
1806 3000
1807 3000
1808 3000
1809 3000
1810 3000
1811 3000
1812 3000
1813 3000
1814 3000
1815 3000
1816 3000
1817 3000
1818 3000
1819 3000
1820 3000
1821 3000
1822 3000
1823 3000
1824 3000
1825 3000
1826 3000
1827 3000
1828 3000
1829 3000
1830 3000
1831 3000
1832 3000
1833 3000
1834 3000
1835 3000
1836 3000
1837 3000
1838 3000
1839 3000
1840 3000
1841 3000
1842 3000
1843 3000
1844 3000
1845 3000
1846 3000
1847 3000
1848 3000
1849 3000
1850 3000


2571 3000
2572 3000
2573 3000
2574 3000
2575 3000
2576 3000
2577 3000
2578 3000
2579 3000
2580 3000
2581 3000
2582 3000
2583 3000
2584 3000
2585 3000
2586 3000
2587 3000
2588 3000
2589 3000
2590 3000
2591 3000
2592 3000
2593 3000
2594 3000
2595 3000
2596 3000
2597 3000
2598 3000
2599 3000
2600 3000
2601 3000
2602 3000
2603 3000
2604 3000
2605 3000
2606 3000
2607 3000
2608 3000
2609 3000
2610 3000
2611 3000
2612 3000
2613 3000
2614 3000
2615 3000
2616 3000
2617 3000
2618 3000
2619 3000
2620 3000
2621 3000
2622 3000
2623 3000
2624 3000
2625 3000
2626 3000
2627 3000
2628 3000
2629 3000
2630 3000
2631 3000
2632 3000
2633 3000
2634 3000
2635 3000
2636 3000
2637 3000
2638 3000
2639 3000
2640 3000
2641 3000
2642 3000
2643 3000
2644 3000
2645 3000
2646 3000
2647 3000
2648 3000
2649 3000
2650 3000
2651 3000
2652 3000
2653 3000
2654 3000
2655 3000
2656 3000
2657 3000
2658 3000
2659 3000
2660 3000
2661 3000
2662 3000
2663 3000
2664 3000
2665 3000
2666 3000
2667 3000
2668 3000
2669 3000
2670 3000


3391 3000
3392 3000
3393 3000
3394 3000
3395 3000
3396 3000
3397 3000
3398 3000
3399 3000
3400 3000
3401 3000
3402 3000
3403 3000
3404 3000
3405 3000
3406 3000
3407 3000
3408 3000
3409 3000
3410 3000
3411 3000
3412 3000
3413 3000
3414 3000
3415 3000
3416 3000
3417 3000
3418 3000
3419 3000
3420 3000
3421 3000
3422 3000
3423 3000
3424 3000
3425 3000
3426 3000
3427 3000
3428 3000
3429 3000
3430 3000
3431 3000
3432 3000
3433 3000
3434 3000
3435 3000
3436 3000
3437 3000
3438 3000
3439 3000
3440 3000
3441 3000
3442 3000
3443 3000
3444 3000
3445 3000
3446 3000
3447 3000
3448 3000
3449 3000
3450 3000
3451 3000
3452 3000
3453 3000
3454 3000
3455 3000
3456 3000
3457 3000
3458 3000
3459 3000
3460 3000
3461 3000
3462 3000
3463 3000
3464 3000
3465 3000
3466 3000
3467 3000
3468 3000
3469 3000
3470 3000
3471 3000
3472 3000
3473 3000
3474 3000
3475 3000
3476 3000
3477 3000
3478 3000
3479 3000
3480 3000
3481 3000
3482 3000
3483 3000
3484 3000
3485 3000
3486 3000
3487 3000
3488 3000
3489 3000
3490 3000


...

34547 3000
34548 3000
34549 3000
34550 3000
34551 3000
34552 3000
34553 3000
34554 3000
34555 3000
34556 3000
34557 3000
34558 3000
34559 3000
34560 3000
34561 3000
34562 3000
34563 3000
34564 3000
34565 3000
34566 3000
34567 3000
34568 3000
34569 3000
34570 3000
34571 3000
34572 3000
34573 3000
34574 3000
34575 3000
34576 3000
34577 3000
34578 3000
34579 3000
34580 3000
34581 3000
34582 3000
34583 3000
34584 3000
34585 3000
34586 3000
34587 3000
34588 3000
34589 3000
34590 3000
34591 3000
34592 3000
34593 3000
34594 3000
34595 3000
34596 3000
34597 3000
34598 3000
34599 3000
34600 3000
34601 3000
34602 3000
34603 3000
34604 3000
34605 3000
34606 3000
34607 3000
34608 3000
34609 3000
34610 3000
34611 3000
34612 3000
34613 3000
34614 3000
34615 3000
34616 3000
34617 3000
34618 3000
34619 3000
34620 3000
34621 3000
34622 3000
34623 3000
34624 3000
34625 3000
34626 3000
34627 3000
34628 3000
34629 3000
34630 3000
34631 3000
34632 3000
34633 3000
34634 3000
34635 3000
34636 3000
34637 3000

35292 3000
35293 3000
35294 3000
35295 3000
35296 3000
35297 3000
35298 3000
35299 3000
35300 3000
35301 3000
35302 3000
35303 3000
35304 3000
35305 3000
35306 3000
35307 3000
35308 3000
35309 3000
35310 3000
35311 3000
35312 3000
35313 3000
35314 3000
35315 3000
35316 3000
35317 3000
35318 3000
35319 3000
35320 3000
35321 3000
35322 3000
35323 3000
35324 3000
35325 3000
35326 3000
35327 3000
35328 3000
35329 3000
35330 3000
35331 3000
35332 3000
35333 3000
35334 3000
35335 3000
35336 3000
35337 3000
35338 3000
35339 3000
35340 3000
35341 3000
35342 3000
35343 3000
35344 3000
35345 3000
35346 3000
35347 3000
35348 3000
35349 3000
35350 3000
35351 3000
35352 3000
35353 3000
35354 3000
35355 3000
35356 3000
35357 3000
35358 3000
35359 3000
35360 3000
35361 3000
35362 3000
35363 3000
35364 3000
35365 3000
35366 3000
35367 3000
35368 3000
35369 3000
35370 3000
35371 3000
35372 3000
35373 3000
35374 3000
35375 3000
35376 3000
35377 3000
35378 3000
35379 3000
35380 3000
35381 3000
35382 3000

36037 3000
36038 3000
36039 3000
36040 3000
36041 3000
36042 3000
36043 3000
36044 3000
36045 3000
36046 3000
36047 3000
36048 3000
36049 3000
36050 3000
36051 3000
36052 3000
36053 3000
36054 3000
36055 3000
36056 3000
36057 3000
36058 3000
36059 3000
36060 3000
36061 3000
36062 3000
36063 3000
36064 3000
36065 3000
36066 3000
36067 3000
36068 3000
36069 3000
36070 3000
36071 3000
36072 3000
36073 3000
36074 3000
36075 3000
36076 3000
36077 3000
36078 3000
36079 3000
36080 3000
36081 3000
36082 3000
36083 3000
36084 3000
36085 3000
36086 3000
36087 3000
36088 3000
36089 3000
36090 3000
36091 3000
36092 3000
36093 3000
36094 3000
36095 3000
36096 3000
36097 3000
36098 3000
36099 3000
36100 3000
36101 3000
36102 3000
36103 3000
36104 3000
36105 3000
36106 3000
36107 3000
36108 3000
36109 3000
36110 3000
36111 3000
36112 3000
36113 3000
36114 3000
36115 3000
36116 3000
36117 3000
36118 3000
36119 3000
36120 3000
36121 3000
36122 3000
36123 3000
36124 3000
36125 3000
36126 3000
36127 3000

36782 3000
36783 3000
36784 3000
36785 3000
36786 3000
36787 3000
36788 3000
36789 3000
36790 3000
36791 3000
36792 3000
36793 3000
36794 3000
36795 3000
36796 3000
36797 3000
36798 3000
36799 3000
36800 3000
36801 3000
36802 3000
36803 3000
36804 3000
36805 3000
36806 3000
36807 3000
36808 3000
36809 3000
36810 3000
36811 3000
36812 3000
36813 3000
36814 3000
36815 3000
36816 3000
36817 3000
36818 3000
36819 3000
36820 3000
36821 3000
36822 3000
36823 3000
36824 3000
36825 3000
36826 3000
36827 3000
36828 3000
36829 3000
36830 3000
36831 3000
36832 3000
36833 3000
36834 3000
36835 3000
36836 3000
36837 3000
36838 3000
36839 3000
36840 3000
36841 3000
36842 3000
36843 3000
36844 3000
36845 3000
36846 3000
36847 3000
36848 3000
36849 3000
36850 3000
36851 3000
36852 3000
36853 3000
36854 3000
36855 3000
36856 3000
36857 3000
36858 3000
36859 3000
36860 3000
36861 3000
36862 3000
36863 3000
36864 3000
36865 3000
36866 3000
36867 3000
36868 3000
36869 3000
36870 3000
36871 3000
36872 3000

37527 3000
37528 3000
37529 3000
37530 3000
37531 3000
37532 3000
37533 3000
37534 3000
37535 3000
37536 3000
37537 3000
37538 3000
37539 3000
37540 3000
37541 3000
37542 3000
37543 3000
37544 3000
37545 3000
37546 3000
37547 3000
37548 3000
37549 3000
37550 3000
37551 3000
37552 3000
37553 3000
37554 3000
37555 3000
37556 3000
37557 3000
37558 3000
37559 3000
37560 3000
37561 3000
37562 3000
37563 3000
37564 3000
37565 3000
37566 3000
37567 3000
37568 3000
37569 3000
37570 3000
37571 3000
37572 3000
37573 3000
37574 3000
37575 3000
37576 3000
37577 3000
37578 3000
37579 3000
37580 3000
37581 3000
37582 3000
37583 3000
37584 3000
37585 3000
37586 3000
37587 3000
37588 3000
37589 3000
37590 3000
37591 3000
37592 3000
37593 3000
37594 3000
37595 3000
37596 3000
37597 3000
37598 3000
37599 3000
37600 3000
37601 3000
37602 3000
37603 3000
37604 3000
37605 3000
37606 3000
37607 3000
37608 3000
37609 3000
37610 3000
37611 3000
37612 3000
37613 3000
37614 3000
37615 3000
37616 3000
37617 3000

In [23]:
process_data(2018,dfManZon_array)

0 3000
1 3000
2 3000
3 3000
4 3000
5 3000
6 3000
7 3000
8 3000
9 3000
10 3000
11 3000
12 3000
13 3000
14 3000
15 3000
16 3000
17 3000
18 3000
19 3000
20 3000
21 3000
22 3000
23 3000
24 3000
25 3000
26 3000
27 3000
28 3000
29 3000
30 3000
31 3000
32 3000
33 3000
34 3000
35 3000
36 3000
37 3000
38 3000
39 3000
40 3000
41 3000
42 3000
43 3000
44 3000
45 3000
46 3000
47 3000
48 3000
49 3000
50 3000
51 3000
52 3000
53 3000
54 3000
55 3000
56 3000
57 3000
58 3000
59 3000
60 3000
61 3000
62 3000
63 3000
64 3000
65 3000
66 3000
67 3000
68 3000
69 3000
70 3000
71 3000
72 3000
73 3000
74 3000
75 3000
76 3000
77 3000
78 3000
79 3000
80 3000
81 3000
82 3000
83 3000
84 3000
85 3000
86 3000
87 3000
88 3000
89 3000
90 3000
91 3000
92 3000
93 3000
94 3000
95 3000
96 3000
97 3000
98 3000
99 3000
100 3000
101 3000
102 3000
103 3000
104 3000
105 3000
106 3000
107 3000
108 3000
109 3000
110 3000
111 3000
112 3000
113 3000
114 3000
115 3000
116 3000
117 3000
118 3000
119 3000
120 3000
121 3000
122 3000
123

923 3000
924 3000
925 3000
926 3000
927 3000
928 3000
929 3000
930 3000
931 3000
932 3000
933 3000
934 3000
935 3000
936 3000
937 3000
938 3000
939 3000
940 3000
941 3000
942 3000
943 3000
944 3000
945 3000
946 3000
947 3000
948 3000
949 3000
950 3000
951 3000
952 3000
953 3000
954 3000
955 3000
956 3000
957 3000
958 3000
959 3000
960 3000
961 3000
962 3000
963 3000
964 3000
965 3000
966 3000
967 3000
968 3000
969 3000
970 3000
971 3000
972 3000
973 3000
974 3000
975 3000
976 3000
977 3000
978 3000
979 3000
980 3000
981 3000
982 3000
983 3000
984 3000
985 3000
986 3000
987 3000
988 3000
989 3000
990 3000
991 3000
992 3000
993 3000
994 3000
995 3000
996 3000
997 3000
998 3000
999 3000
1000 3000
1001 3000
1002 3000
1003 3000
1004 3000
1005 3000
1006 3000
1007 3000
1008 3000
1009 3000
1010 3000
1011 3000
1012 3000
1013 3000
1014 3000
1015 3000
1016 3000
1017 3000
1018 3000
1019 3000
1020 3000
1021 3000
1022 3000
1023 3000
1024 3000
1025 3000
1026 3000
1027 3000
1028 3000
1029 3000
1030 30

1750 3000
1751 3000
1752 3000
1753 3000
1754 3000
1755 3000
1756 3000
1757 3000
1758 3000
1759 3000
1760 3000
1761 3000
1762 3000
1763 3000
1764 3000
1765 3000
1766 3000
1767 3000
1768 3000
1769 3000
1770 3000
1771 3000
1772 3000
1773 3000
1774 3000
1775 3000
1776 3000
1777 3000
1778 3000
1779 3000
1780 3000
1781 3000
1782 3000
1783 3000
1784 3000
1785 3000
1786 3000
1787 3000
1788 3000
1789 3000
1790 3000
1791 3000
1792 3000
1793 3000
1794 3000
1795 3000
1796 3000
1797 3000
1798 3000
1799 3000
1800 3000
1801 3000
1802 3000
1803 3000
1804 3000
1805 3000
1806 3000
1807 3000
1808 3000
1809 3000
1810 3000
1811 3000
1812 3000
1813 3000
1814 3000
1815 3000
1816 3000
1817 3000
1818 3000
1819 3000
1820 3000
1821 3000
1822 3000
1823 3000
1824 3000
1825 3000
1826 3000
1827 3000
1828 3000
1829 3000
1830 3000
1831 3000
1832 3000
1833 3000
1834 3000
1835 3000
1836 3000
1837 3000
1838 3000
1839 3000
1840 3000
1841 3000
1842 3000
1843 3000
1844 3000
1845 3000
1846 3000
1847 3000
1848 3000
1849 3000


2570 3000
2571 3000
2572 3000
2573 3000
2574 3000
2575 3000
2576 3000
2577 3000
2578 3000
2579 3000
2580 3000
2581 3000
2582 3000
2583 3000
2584 3000
2585 3000
2586 3000
2587 3000
2588 3000
2589 3000
2590 3000
2591 3000
2592 3000
2593 3000
2594 3000
2595 3000
2596 3000
2597 3000
2598 3000
2599 3000
2600 3000
2601 3000
2602 3000
2603 3000
2604 3000
2605 3000
2606 3000
2607 3000
2608 3000
2609 3000
2610 3000
2611 3000
2612 3000
2613 3000
2614 3000
2615 3000
2616 3000
2617 3000
2618 3000
2619 3000
2620 3000
2621 3000
2622 3000
2623 3000
2624 3000
2625 3000
2626 3000
2627 3000
2628 3000
2629 3000
2630 3000
2631 3000
2632 3000
2633 3000
2634 3000
2635 3000
2636 3000
2637 3000
2638 3000
2639 3000
2640 3000
2641 3000
2642 3000
2643 3000
2644 3000
2645 3000
2646 3000
2647 3000
2648 3000
2649 3000
2650 3000
2651 3000
2652 3000
2653 3000
2654 3000
2655 3000
2656 3000
2657 3000
2658 3000
2659 3000
2660 3000
2661 3000
2662 3000
2663 3000
2664 3000
2665 3000
2666 3000
2667 3000
2668 3000
2669 3000


3390 3000
3391 3000
3392 3000
3393 3000
3394 3000
3395 3000
3396 3000
3397 3000
3398 3000
3399 3000
3400 3000
3401 3000
3402 3000
3403 3000
3404 3000
3405 3000
3406 3000
3407 3000
3408 3000
3409 3000
3410 3000
3411 3000
3412 3000
3413 3000
3414 3000
3415 3000
3416 3000
3417 3000
3418 3000
3419 3000
3420 3000
3421 3000
3422 3000
3423 3000
3424 3000
3425 3000
3426 3000
3427 3000
3428 3000
3429 3000
3430 3000
3431 3000
3432 3000
3433 3000
3434 3000
3435 3000
3436 3000
3437 3000
3438 3000
3439 3000
3440 3000
3441 3000
3442 3000
3443 3000
3444 3000
3445 3000
3446 3000
3447 3000
3448 3000
3449 3000
3450 3000
3451 3000
3452 3000
3453 3000
3454 3000
3455 3000
3456 3000
3457 3000
3458 3000
3459 3000
3460 3000
3461 3000
3462 3000
3463 3000
3464 3000
3465 3000
3466 3000
3467 3000
3468 3000
3469 3000
3470 3000
3471 3000
3472 3000
3473 3000
3474 3000
3475 3000
3476 3000
3477 3000
3478 3000
3479 3000
3480 3000
3481 3000
3482 3000
3483 3000
3484 3000
3485 3000
3486 3000
3487 3000
3488 3000
3489 3000


...

33795 3000
33796 3000
33797 3000
33798 3000
33799 3000
33800 3000
33801 3000
33802 3000
33803 3000
33804 3000
33805 3000
33806 3000
33807 3000
33808 3000
33809 3000
33810 3000
33811 3000
33812 3000
33813 3000
33814 3000
33815 3000
33816 3000
33817 3000
33818 3000
33819 3000
33820 3000
33821 3000
33822 3000
33823 3000
33824 3000
33825 3000
33826 3000
33827 3000
33828 3000
33829 3000
33830 3000
33831 3000
33832 3000
33833 3000
33834 3000
33835 3000
33836 3000
33837 3000
33838 3000
33839 3000
33840 3000
33841 3000
33842 3000
33843 3000
33844 3000
33845 3000
33846 3000
33847 3000
33848 3000
33849 3000
33850 3000
33851 3000
33852 3000
33853 3000
33854 3000
33855 3000
33856 3000
33857 3000
33858 3000
33859 3000
33860 3000
33861 3000
33862 3000
33863 3000
33864 3000
33865 3000
33866 3000
33867 3000
33868 3000
33869 3000
33870 3000
33871 3000
33872 3000
33873 3000
33874 3000
33875 3000
33876 3000
33877 3000
33878 3000
33879 3000
33880 3000
33881 3000
33882 3000
33883 3000
33884 3000
33885 3000

34540 3000
34541 3000
34542 3000
34543 3000
34544 3000
34545 3000
34546 3000
34547 3000
34548 3000
34549 3000
34550 3000
34551 3000
34552 3000
34553 3000
34554 3000
34555 3000
34556 3000
34557 3000
34558 3000
34559 3000
34560 3000
34561 3000
34562 3000
34563 3000
34564 3000
34565 3000
34566 3000
34567 3000
34568 3000
34569 3000
34570 3000
34571 3000
34572 3000
34573 3000
34574 3000
34575 3000
34576 3000
34577 3000
34578 3000
34579 3000
34580 3000
34581 3000
34582 3000
34583 3000
34584 3000
34585 3000
34586 3000
34587 3000
34588 3000
34589 3000
34590 3000
34591 3000
34592 3000
34593 3000
34594 3000
34595 3000
34596 3000
34597 3000
34598 3000
34599 3000
34600 3000
34601 3000
34602 3000
34603 3000
34604 3000
34605 3000
34606 3000
34607 3000
34608 3000
34609 3000
34610 3000
34611 3000
34612 3000
34613 3000
34614 3000
34615 3000
34616 3000
34617 3000
34618 3000
34619 3000
34620 3000
34621 3000
34622 3000
34623 3000
34624 3000
34625 3000
34626 3000
34627 3000
34628 3000
34629 3000
34630 3000

35285 3000
35286 3000
35287 3000
35288 3000
35289 3000
35290 3000
35291 3000
35292 3000
35293 3000
35294 3000
35295 3000
35296 3000
35297 3000
35298 3000
35299 3000
35300 3000
35301 3000
35302 3000
35303 3000
35304 3000
35305 3000
35306 3000
35307 3000
35308 3000
35309 3000
35310 3000
35311 3000
35312 3000
35313 3000
35314 3000
35315 3000
35316 3000
35317 3000
35318 3000
35319 3000
35320 3000
35321 3000
35322 3000
35323 3000
35324 3000
35325 3000
35326 3000
35327 3000
35328 3000
35329 3000
35330 3000
35331 3000
35332 3000
35333 3000
35334 3000
35335 3000
35336 3000
35337 3000
35338 3000
35339 3000
35340 3000
35341 3000
35342 3000
35343 3000
35344 3000
35345 3000
35346 3000
35347 3000
35348 3000
35349 3000
35350 3000
35351 3000
35352 3000
35353 3000
35354 3000
35355 3000
35356 3000
35357 3000
35358 3000
35359 3000
35360 3000
35361 3000
35362 3000
35363 3000
35364 3000
35365 3000
35366 3000
35367 3000
35368 3000
35369 3000
35370 3000
35371 3000
35372 3000
35373 3000
35374 3000
35375 3000

36030 3000
36031 3000
36032 3000
36033 3000
36034 3000
36035 3000
36036 3000
36037 3000
36038 3000
36039 3000
36040 3000
36041 3000
36042 3000
36043 3000
36044 3000
36045 3000
36046 3000
36047 3000
36048 3000
36049 3000
36050 3000
36051 3000
36052 3000
36053 3000
36054 3000
36055 3000
36056 3000
36057 3000
36058 3000
36059 3000
36060 3000
36061 3000
36062 3000
36063 3000
36064 3000
36065 3000
36066 3000
36067 3000
36068 3000
36069 3000
36070 3000
36071 3000
36072 3000
36073 3000
36074 3000
36075 3000
36076 3000
36077 3000
36078 3000
36079 3000
36080 3000
36081 3000
36082 3000
36083 3000
36084 3000
36085 3000
36086 3000
36087 3000
36088 3000
36089 3000
36090 3000
36091 3000
36092 3000
36093 3000
36094 3000
36095 3000
36096 3000
36097 3000
36098 3000
36099 3000
36100 3000
36101 3000
36102 3000
36103 3000
36104 3000
36105 3000
36106 3000
36107 3000
36108 3000
36109 3000
36110 3000
36111 3000
36112 3000
36113 3000
36114 3000
36115 3000
36116 3000
36117 3000
36118 3000
36119 3000
36120 3000

36775 3000
36776 3000
36777 3000
36778 3000
36779 3000
36780 3000
36781 3000
36782 3000
36783 3000
36784 3000
36785 3000
36786 3000
36787 3000
36788 3000
36789 3000
36790 3000
36791 3000
36792 3000
36793 3000
36794 3000
36795 3000
36796 3000
36797 3000
36798 3000
36799 3000
36800 3000
36801 3000
36802 3000
36803 3000
36804 3000
36805 3000
36806 3000
36807 3000
36808 3000
36809 3000
36810 3000
36811 3000
36812 3000
36813 3000
36814 3000
36815 3000
36816 3000
36817 3000
36818 3000
36819 3000
36820 3000
36821 3000
36822 3000
36823 3000
36824 3000
36825 3000
36826 3000
36827 3000
36828 3000
36829 3000
36830 3000
36831 3000
36832 3000
36833 3000
36834 3000
36835 3000
36836 3000
36837 3000
36838 3000
36839 3000
36840 3000
36841 3000
36842 3000
36843 3000
36844 3000
36845 3000
36846 3000
36847 3000
36848 3000
36849 3000
36850 3000
36851 3000
36852 3000
36853 3000
36854 3000
36855 3000
36856 3000
36857 3000
36858 3000
36859 3000
36860 3000
36861 3000
36862 3000
36863 3000
36864 3000
36865 3000

In [24]:
process_data(2019,dfManZon_array)

0 3000
1 3000
2 3000
3 3000
4 3000
5 3000
6 3000
7 3000
8 3000
9 3000
10 3000
11 3000
12 3000
13 3000
14 3000
15 3000
16 3000
17 3000
18 3000
19 3000
20 3000
21 3000
22 3000
23 3000
24 3000
25 3000
26 3000
27 3000
28 3000
29 3000
30 3000
31 3000
32 3000
33 3000
34 3000
35 3000
36 3000
37 3000
38 3000
39 3000
40 3000
41 3000
42 3000
43 3000
44 3000
45 3000
46 3000
47 3000
48 3000
49 3000
50 3000
51 3000
52 3000
53 3000
54 3000
55 3000
56 3000
57 3000
58 3000
59 3000
60 3000
61 3000
62 3000
63 3000
64 3000
65 3000
66 3000
67 3000
68 3000
69 3000
70 3000
71 3000
72 3000
73 3000
74 3000
75 3000
76 3000
77 3000
78 3000
79 3000
80 3000
81 3000
82 3000
83 3000
84 3000
85 3000
86 3000
87 3000
88 3000
89 3000
90 3000
91 3000
92 3000
93 3000
94 3000
95 3000
96 3000
97 3000
98 3000
99 3000
100 3000
101 3000
102 3000
103 3000
104 3000
105 3000
106 3000
107 3000
108 3000
109 3000
110 3000
111 3000
112 3000
113 3000
114 3000
115 3000
116 3000
117 3000
118 3000
119 3000
120 3000
121 3000
122 3000
123

923 3000
924 3000
925 3000
926 3000
927 3000
928 3000
929 3000
930 3000
931 3000
932 3000
933 3000
934 3000
935 3000
936 3000
937 3000
938 3000
939 3000
940 3000
941 3000
942 3000
943 3000
944 3000
945 3000
946 3000
947 3000
948 3000
949 3000
950 3000
951 3000
952 3000
953 3000
954 3000
955 3000
956 3000
957 3000
958 3000
959 3000
960 3000
961 3000
962 3000
963 3000
964 3000
965 3000
966 3000
967 3000
968 3000
969 3000
970 3000
971 3000
972 3000
973 3000
974 3000
975 3000
976 3000
977 3000
978 3000
979 3000
980 3000
981 3000
982 3000
983 3000
984 3000
985 3000
986 3000
987 3000
988 3000
989 3000
990 3000
991 3000
992 3000
993 3000
994 3000
995 3000
996 3000
997 3000
998 3000
999 3000
1000 3000
1001 3000
1002 3000
1003 3000
1004 3000
1005 3000
1006 3000
1007 3000
1008 3000
1009 3000
1010 3000
1011 3000
1012 3000
1013 3000
1014 3000
1015 3000
1016 3000
1017 3000
1018 3000
1019 3000
1020 3000
1021 3000
1022 3000
1023 3000
1024 3000
1025 3000
1026 3000
1027 3000
1028 3000
1029 3000
1030 30

1750 3000
1751 3000
1752 3000
1753 3000
1754 3000
1755 3000
1756 3000
1757 3000
1758 3000
1759 3000
1760 3000
1761 3000
1762 3000
1763 3000
1764 3000
1765 3000
1766 3000
1767 3000
1768 3000
1769 3000
1770 3000
1771 3000
1772 3000
1773 3000
1774 3000
1775 3000
1776 3000
1777 3000
1778 3000
1779 3000
1780 3000
1781 3000
1782 3000
1783 3000
1784 3000
1785 3000
1786 3000
1787 3000
1788 3000
1789 3000
1790 3000
1791 3000
1792 3000
1793 3000
1794 3000
1795 3000
1796 3000
1797 3000
1798 3000
1799 3000
1800 3000
1801 3000
1802 3000
1803 3000
1804 3000
1805 3000
1806 3000
1807 3000
1808 3000
1809 3000
1810 3000
1811 3000
1812 3000
1813 3000
1814 3000
1815 3000
1816 3000
1817 3000
1818 3000
1819 3000
1820 3000
1821 3000
1822 3000
1823 3000
1824 3000
1825 3000
1826 3000
1827 3000
1828 3000
1829 3000
1830 3000
1831 3000
1832 3000
1833 3000
1834 3000
1835 3000
1836 3000
1837 3000
1838 3000
1839 3000
1840 3000
1841 3000
1842 3000
1843 3000
1844 3000
1845 3000
1846 3000
1847 3000
1848 3000
1849 3000


2570 3000
2571 3000
2572 3000
2573 3000
2574 3000
2575 3000
2576 3000
2577 3000
2578 3000
2579 3000
2580 3000
2581 3000
2582 3000
2583 3000
2584 3000
2585 3000
2586 3000
2587 3000
2588 3000
2589 3000
2590 3000
2591 3000
2592 3000
2593 3000
2594 3000
2595 3000
2596 3000
2597 3000
2598 3000
2599 3000
2600 3000
2601 3000
2602 3000
2603 3000
2604 3000
2605 3000
2606 3000
2607 3000
2608 3000
2609 3000
2610 3000
2611 3000
2612 3000
2613 3000
2614 3000
2615 3000
2616 3000
2617 3000
2618 3000
2619 3000
2620 3000
2621 3000
2622 3000
2623 3000
2624 3000
2625 3000
2626 3000
2627 3000
2628 3000
2629 3000
2630 3000
2631 3000
2632 3000
2633 3000
2634 3000
2635 3000
2636 3000
2637 3000
2638 3000
2639 3000
2640 3000
2641 3000
2642 3000
2643 3000
2644 3000
2645 3000
2646 3000
2647 3000
2648 3000
2649 3000
2650 3000
2651 3000
2652 3000
2653 3000
2654 3000
2655 3000
2656 3000
2657 3000
2658 3000
2659 3000
2660 3000
2661 3000
2662 3000
2663 3000
2664 3000
2665 3000
2666 3000
2667 3000
2668 3000
2669 3000


3390 3000
3391 3000
3392 3000
3393 3000
3394 3000
3395 3000
3396 3000
3397 3000
3398 3000
3399 3000
3400 3000
3401 3000
3402 3000
3403 3000
3404 3000
3405 3000
3406 3000
3407 3000
3408 3000
3409 3000
3410 3000
3411 3000
3412 3000
3413 3000
3414 3000
3415 3000
3416 3000
3417 3000
3418 3000
3419 3000
3420 3000
3421 3000
3422 3000
3423 3000
3424 3000
3425 3000
3426 3000
3427 3000
3428 3000
3429 3000
3430 3000
3431 3000
3432 3000
3433 3000
3434 3000
3435 3000
3436 3000
3437 3000
3438 3000
3439 3000
3440 3000
3441 3000
3442 3000
3443 3000
3444 3000
3445 3000
3446 3000
3447 3000
3448 3000
3449 3000
3450 3000
3451 3000
3452 3000
3453 3000
3454 3000
3455 3000
3456 3000
3457 3000
3458 3000
3459 3000
3460 3000
3461 3000
3462 3000
3463 3000
3464 3000
3465 3000
3466 3000
3467 3000
3468 3000
3469 3000
3470 3000
3471 3000
3472 3000
3473 3000
3474 3000
3475 3000
3476 3000
3477 3000
3478 3000
3479 3000
3480 3000
3481 3000
3482 3000
3483 3000
3484 3000
3485 3000
3486 3000
3487 3000
3488 3000
3489 3000


...

24855 3000
24856 3000
24857 3000
24858 3000
24859 3000
24860 3000
24861 3000
24862 3000
24863 3000
24864 3000
24865 3000
24866 3000
24867 3000
24868 3000
24869 3000
24870 3000
24871 3000
24872 3000
24873 3000
24874 3000
24875 3000
24876 3000
24877 3000
24878 3000
24879 3000
24880 3000
24881 3000
24882 3000
24883 3000
24884 3000
24885 3000
24886 3000
24887 3000
24888 3000
24889 3000
24890 3000
24891 3000
24892 3000
24893 3000
24894 3000
24895 3000
24896 3000
24897 3000
24898 3000
24899 3000
24900 3000
24901 3000
24902 3000
24903 3000
24904 3000
24905 3000
24906 3000
24907 3000
24908 3000
24909 3000
24910 3000
24911 3000
24912 3000
24913 3000
24914 3000
24915 3000
24916 3000
24917 3000
24918 3000
24919 3000
24920 3000
24921 3000
24922 3000
24923 3000
24924 3000
24925 3000
24926 3000
24927 3000
24928 3000
24929 3000
24930 3000
24931 3000
24932 3000
24933 3000
24934 3000
24935 3000
24936 3000
24937 3000
24938 3000
24939 3000
24940 3000
24941 3000
24942 3000
24943 3000
24944 3000
24945 3000

25600 3000
25601 3000
25602 3000
25603 3000
25604 3000
25605 3000
25606 3000
25607 3000
25608 3000
25609 3000
25610 3000
25611 3000
25612 3000
25613 3000
25614 3000
25615 3000
25616 3000
25617 3000
25618 3000
25619 3000
25620 3000
25621 3000
25622 3000
25623 3000
25624 3000
25625 3000
25626 3000
25627 3000
25628 3000
25629 3000
25630 3000
25631 3000
25632 3000
25633 3000
25634 3000
25635 3000
25636 3000
25637 3000
25638 3000
25639 3000
25640 3000
25641 3000
25642 3000
25643 3000
25644 3000
25645 3000
25646 3000
25647 3000
25648 3000
25649 3000
25650 3000
25651 3000
25652 3000
25653 3000
25654 3000
25655 3000
25656 3000
25657 3000
25658 3000
25659 3000
25660 3000
25661 3000
25662 3000
25663 3000
25664 3000
25665 3000
25666 3000
25667 3000
25668 3000
25669 3000
25670 3000
25671 3000
25672 3000
25673 3000
25674 3000
25675 3000
25676 3000
25677 3000
25678 3000
25679 3000
25680 3000
25681 3000
25682 3000
25683 3000
25684 3000
25685 3000
25686 3000
25687 3000
25688 3000
25689 3000
25690 3000

26345 3000
26346 3000
26347 3000
26348 3000
26349 3000
26350 3000
26351 3000
26352 3000
26353 3000
26354 3000
26355 3000
26356 3000
26357 3000
26358 3000
26359 3000
26360 3000
26361 3000
26362 3000
26363 3000
26364 3000
26365 3000
26366 3000
26367 3000
26368 3000
26369 3000
26370 3000
26371 3000
26372 3000
26373 3000
26374 3000
26375 3000
26376 3000
26377 3000
26378 3000
26379 3000
26380 3000
26381 3000
26382 3000
26383 3000
26384 3000
26385 3000
26386 3000
26387 3000
26388 3000
26389 3000
26390 3000
26391 3000
26392 3000
26393 3000
26394 3000
26395 3000
26396 3000
26397 3000
26398 3000
26399 3000
26400 3000
26401 3000
26402 3000
26403 3000
26404 3000
26405 3000
26406 3000
26407 3000
26408 3000
26409 3000
26410 3000
26411 3000
26412 3000
26413 3000
26414 3000
26415 3000
26416 3000
26417 3000
26418 3000
26419 3000
26420 3000
26421 3000
26422 3000
26423 3000
26424 3000
26425 3000
26426 3000
26427 3000
26428 3000
26429 3000
26430 3000
26431 3000
26432 3000
26433 3000
26434 3000
26435 3000

27090 3000
27091 3000
27092 3000
27093 3000
27094 3000
27095 3000
27096 3000
27097 3000
27098 3000
27099 3000
27100 3000
27101 3000
27102 3000
27103 3000
27104 3000
27105 3000
27106 3000
27107 3000
27108 3000
27109 3000
27110 3000
27111 3000
27112 3000
27113 3000
27114 3000
27115 3000
27116 3000
27117 3000
27118 3000
27119 3000
27120 3000
27121 3000
27122 3000
27123 3000
27124 3000
27125 3000
27126 3000
27127 3000
27128 3000
27129 3000
27130 3000
27131 3000
27132 3000
27133 3000
27134 3000
27135 3000
27136 3000
27137 3000
27138 3000
27139 3000
27140 3000
27141 3000
27142 3000
27143 3000
27144 3000
27145 3000
27146 3000
27147 3000
27148 3000
27149 3000
27150 3000
27151 3000
27152 3000
27153 3000
27154 3000
27155 3000
27156 3000
27157 3000
27158 3000
27159 3000
27160 3000
27161 3000
27162 3000
27163 3000
27164 3000
27165 3000
27166 3000
27167 3000
27168 3000
27169 3000
27170 3000
27171 3000
27172 3000
27173 3000
27174 3000
27175 3000
27176 3000
27177 3000
27178 3000
27179 3000
27180 3000

27835 3000
27836 3000
27837 3000
27838 3000
27839 3000
27840 3000
27841 3000
27842 3000
27843 3000
27844 3000
27845 3000
27846 3000
27847 3000
27848 3000
27849 3000
27850 3000
27851 3000
27852 3000
27853 3000
27854 3000
27855 3000
27856 3000
27857 3000
27858 3000
27859 3000
27860 3000
27861 3000
27862 3000
27863 3000
27864 3000
27865 3000
27866 3000
27867 3000
27868 3000
27869 3000
27870 3000
27871 3000
27872 3000
27873 3000
27874 3000
27875 3000
27876 3000
27877 3000
27878 3000
27879 3000
27880 3000
27881 3000
27882 3000
27883 3000
27884 3000
27885 3000
27886 3000
27887 3000
27888 3000
27889 3000
27890 3000
27891 3000
27892 3000
27893 3000
27894 3000
27895 3000
27896 3000
27897 3000
27898 3000
27899 3000
27900 3000
27901 3000
27902 3000
27903 3000
27904 3000
27905 3000
27906 3000
27907 3000
27908 3000
27909 3000
27910 3000
27911 3000
27912 3000
27913 3000
27914 3000
27915 3000
27916 3000
27917 3000
27918 3000
27919 3000
27920 3000
27921 3000
27922 3000
27923 3000
27924 3000
27925 3000

In [24]:
# PERSONAL NOTES:
# interpolate missing values every hour

### 4. Sanity Check
Let´s check that the csv file created is correct

In [22]:
year=2017

csvcheck = pd.read_csv('../data/Data_Taxis_'+str(year)+'_Cleaned.csv', sep=',',
                       parse_dates=['pickup_datetime'])

print('Year should be ' + str(year)+':', csvcheck.year.unique())
print('There should be 12 months:', csvcheck.month.nunique())
print('There should be 31 days:', csvcheck.day.nunique())
print('There should be 24 hours:', csvcheck.hour.nunique())
print('There should be 52 weeks:', csvcheck.week.nunique())
print('There should be 67 UNIQUE LocationIDs',pd.unique(csvcheck['LocationID']).shape)

csvcheck['hourlyperiods'] = 1
h = csvcheck.groupby(['month','day','hour'])['hourlyperiods'].sum()
print('There should be 8760 hourly periods in a year: ', h.shape)
print(csvcheck.describe())

Year should be 2017: [2017]
There should be 12 months: 12
There should be 31 days: 31
There should be 24 hours: 24
There should be 52 weeks: 52
There should be 67 UNIQUE LocationIDs (67,)
There should be 8760 hourly periods in a year:  (8760,)
          LocationID    NoOfPickups      year          month            day  \
count  536306.000000  536306.000000  536306.0  536306.000000  536306.000000   
mean      149.504078     191.191730    2017.0       6.520846      15.718351   
std        74.727735     207.806338       0.0       3.445286       8.794833   
min         4.000000       1.000000    2017.0       1.000000       1.000000   
25%        88.000000      31.000000    2017.0       4.000000       8.000000   
50%       151.000000     116.000000    2017.0       7.000000      16.000000   
75%       230.000000     290.000000    2017.0      10.000000      23.000000   
max       263.000000    3280.000000    2017.0      12.000000      31.000000   

                hour           week      day